# Okamoto-Tanaka Revisited protocol


In [1]:
import sys
print(sys.version)

3.5.3 (default, Sep 27 2018, 17:25:39) 
[GCC 6.3.0 20170516]


In [2]:
import random
import os
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import math

KEY_SIZE = 1024
H1_EXPONENT_SIZE = 256
SESSION_KEY_SIZE = 128
EPHEMERAL_EXPONENT_SIZE = 256
ID_SIZE = 64
ENDIAN = "little"


In [3]:
#dobór funkcji haschującej
def select_SHA_context(length_in_bits):
    if length_in_bits <= 224:
        return hashes.SHA224()
    elif length_in_bits <= 256:
        return hashes.SHA256()
    elif length_in_bits <= 384:
        return hashes.SHA384()
    elif length_in_bits <= 512:
        return hashes.SHA512()
    else:
        #TODO Exception
        print("Error, output to big for now")

#hf_KGC = select_SHA_context(H1_EXPONENT_SIZE)
#hf_USER = select_SHA_context(SESSION_KEY_SIZE)


In [4]:
def eea(_a,_b):
    u1 = 1
    v1 = 0
    u2 = 0
    v2 = 1
    if _a < 1 or _b < 1:
        return "Error"
    
    if _b>_a:
        a = _b
        b = _a
        reverse = False
    else:
        a = _a
        b = _b
        reverse  = True
    r = _b
    while r!=0:
        g = r
        q = a//b
        
        r = a - (q*b)   
        u = u1 - (q*u2)
        v = v1 - (q*v2)
        u1 = u2
        v1 = v2
        u2 = u
        v2 = v
        
        a = b
        b = r
        
        #print(q,r,u,v)
    if reverse:
        return (a, u1, v1)
    else:
        return (a,v1,u1)
    #return (a,v1,u1)


print(eea(77, 11))
print(eea(11, 77))
#print(eea(62, 7))

(11, 0, 1)
(11, 1, 0)


## Generacja klucza



### Pseudolosowość

#### Generacja liczby pseudolosowej przez system (urandom) - kryptograficznie zalecane:

In [5]:
def get_random(num_of_bytes):
    return int.from_bytes(os.urandom(num_of_bytes), byteorder='little')


#### Rozkład x = d * (2^r) 
Najpierw wyznaczamy rozkład wartości funkcji Eulera dla _modułu klucza RSA_ __KGC__. Chcemy znaleźć takie __t__ oraz __r__, że __x = t * (2^r)__, przy czym __t__ jest nieparzyste.

In [6]:
#Rozłożenie N-1 = d* (2**r)
def mr(number):
    t = number
    r = 0
    while t%2 == 0:
    #while math.fmod(d,2) ==0:
        t//=2
        r+=1
    return (t,r)


#### Test Millera-Rabina

In [7]:
def miller_rabin_test(number, security=100):
    nm1 = number-1
    t,r = mr(nm1)
    for i in range(security):
        a = 0
        while a <2 :
            a = get_random(number.bit_length()//8+1)%number
        x = pow(a,t,number)
        if x != 1 and x != nm1:
            pt = False
            for j in range(r):
                x = pow(x,2,number)
                if x == nm1:
                    pt = True
                    break
            if pt == False:
                #witness
                #print(a, t,r)
                return False
    return True
    
miller_rabin_test(21,100)

False

#### Bezpieczne liczby pierwsze:

In [8]:
def get_secure_prime(num_of_bytes, security_level):
    failure_counter = 0
    next_failure_jump = 4
    success = False
    while success == False:
        p1 = get_random(num_of_bytes//2)
        if p1 % 2 == 0:
            p1 += 1
        if miller_rabin_test(p1, security_level) and miller_rabin_test((2*p1)+1, security_level):
            return (2*p1)+1
        else:
            failure_counter +=1
            if failure_counter >= next_failure_jump:
                security_level+=1
                next_failure_jump*=2
#p = get_secure_prime(64, 100)
#q = get_secure_prime(64, 100)

#print(p)
#print(q)
        

### Klasa KGC:

In [9]:
class KGC_key_public:
    def __init__(self, N, e, QRN_generator, h1sf, h2f, key_size):
        self.N = N
        self.e = e
        self.QRN_generator = QRN_generator
        self.h1sf = h1sf
        self.h2f = h2f
        self.key_size = key_size

class KGC_key:
    def __init__(self, h1sf, h2f,key_size = KEY_SIZE):
        
        #rsa_key = rsa.generate_private_key(
        #    public_exponent=65537,
        #    key_size = key_size,
        #    backend = default_backend()
        #)
        self.key_size = key_size
        #self.N = rsa_key.public_key().public_numbers().n
        #self.e = rsa_key.public_key().public_numbers().e
        #self.d = rsa_key.private_numbers().d
        #self.p = rsa_key.private_numbers().p
        #self.q = rsa_key.private_numbers().q
        self.p = get_secure_prime(key_size//16, 100)
        self.q = get_secure_prime(key_size//16, 100)
        self.N = self.p*self.q
        self.e = 65537
        
        self.Phi = (self.p-1)*(self.q-1)
        
        self.d = eea(self.Phi, self.e)[2]
        while self.d < 0:
            self.d += self.Phi#
        #print("Phi:\t", self.Phi)
        #print("e:\t", self.e)
        #print("d:\t", self.d)
        #print("N:\t", self.N)
        #print("eea:\t",eea(self.Phi, self.e))
        #print("p:\t", self.p)
        #print("q:\t", self.q)
        self.h1sf = h1sf # część funkcji hashującej H (wynik całej H musi być w QRN)
        self.h2f = h2f #druga funkcja hashująca jest w całości
    def set_QRN_generator(self, generator):
        self.QRN_generator = generator
    def public(self):
        #return (self.N,self.e,self.generator, self.h1sf, self.h2f)
        return KGC_key_public(self.N, self.e, self.QRN_generator, self.h1sf, self.h2f, self.key_size)
    # d,Phi - traktować jak prywatne

hash1_subfunction = select_SHA_context(H1_EXPONENT_SIZE)
hash2_function = select_SHA_context(SESSION_KEY_SIZE)
    

        

In [19]:
def hashH1(material, kgc_public, endian = ENDIAN, id_size = ID_SIZE):
    #DEBUG_START
    print("hashH1: number to hash")
    print('{:08x}'.format(material))
    material = material.to_bytes(id_size//8, endian)
    #tp = [ord(material[i]) for i in range(material)]
    print("hashH1: bytes to hash:")
    print(material.hex())
    #print(tp)
    
    
    context = hashes.Hash(kgc_public.h1sf, backend=default_backend()) 
    context.update(material)
    hash_result = context.finalize()
    #exponent = int.from_bytes(hash_result, byteorder = endian)
    #print('Hash result:')
    #print (hash_result.hex())
    #print ('{:032x}'.format(exponent))
    h_result = int.from_bytes(hash_result, byteorder = endian)
    print('h_result:\t','{:032x}'.format(h_result))
    #DEBUG_END
    #hsh = pow(kgc_public.QRN_generator, exponent, kgc_public.N)
    hsh = pow(h_result, 2, kgc_public.N)
    print('hsh1:\t','{:032x}'.format(hsh))
    return hsh


### Generacja kluczy uczestników mOT
#### Znalezienie generatora g grupy reszt kwadratowych mod N
Chcemy znaleźć generator podgrupy reszt kwadratowych w grupue Z_N. Oczekujemy, że grupa reszt będzie odpowiednio duża. Podgrupę będziemy oznaczać __QRN__ (analogicznie do oznaczenia w pracy Gannaro-Krawczyka-Rabina), generator będzie oznaczany __g__.
___


##### Wyznaczenie generatora g grupy QR_N
W celu wygenerowania akceptowalnego g losujemy pewnwe __alpha__ i sprawdzamy czy generuje podgrupę o satysfakcjonująco dużym rzędzie. Załóżmy tutaj, że zadowala nas jeżeli __alpha__ podniesione do potęgi t modulo N jest różne od 1.
W dalszej części podnosilibyśmy do kwadratu maksymalnie r razy. Jeżeli __alpha__ nie spełnia oczekiwań losujemy je ponownie i powtarzamy procedurę.  
_W przyszłości rozważyć szukanie większych podgrup (podnieść kilka razy do kwadratu i sprawdzić po ilu podniesieniach jest 1)._

### Tworzenie KGC
Tworzymy instancje KGC upewniając się, że parametry są poprawne i bezpieczne


In [11]:
def valid_kgc():  
    kgc = KGC_key(hash1_subfunction, hash2_function)
    print('{:0128x}'.format(kgc.N))
    print('{:0128x}'.format(kgc.e))
    print('{:0128x}'.format(kgc.d))
    print(hex(kgc.N))
    return kgc
kgc01 = valid_kgc()
#(t,r) = mr(kgc01.Phi)

#print (kgc01.p-1//2)
#print (kgc01.q-1//2)
#rp1 = mr(kgc01.p-1)
#rp2 = mr(kgc01.q-1)
#print('rp1\t', rp1)
#print('rp2\t', rp2)

#print(t,r)
print(pow(pow(2,kgc01.e,kgc01.N),kgc01.d, kgc01.N))
#print(t*(2**r))
#print(t,r)

c1848a104e7e97f6c703a600c68dfca62c94813ac8301e3975174bdb35d9cd089c87a9b648ef1ad4798206f03685100384add639eaf79289cb66caab003e9069
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010001
52d3798c352c75ba47bfc6033af87694612c468b743734187f87330e65e40c9ec07c29d61d92c4586f4d5fb63e818f73349587e664f64aeb90f850a98b1cc0c9
0xc1848a104e7e97f6c703a600c68dfca62c94813ac8301e3975174bdb35d9cd089c87a9b648ef1ad4798206f03685100384add639eaf79289cb66caab003e9069
2


In [21]:
#generator grupy Z_N: Znaleźć alpha że g = alpha**2 jest generatorem dużej podgrupy (reszt kwadratowych)
cont = True
(t,r) = mr(kgc01.Phi)
while cont == True:
    alpha = random.randint(1,kgc01.N)
    #print (pow(alpha, Phi, N))
    beta = pow(alpha,t,kgc01.N)
    print (beta, alpha)
    if  beta != 1:
        cont = False
        #print ("alpha jest generatorem dużej podgrupy")
    
g = pow(alpha,2,kgc01.N)
kgc01.set_QRN_generator(g)
#print (pow(kgc01.QRN_generator, kgc01.Phi, kgc01.N))
print(hex(g))

1390411035702276261406313151674738766539394219797740766624310058164563021918173451563410443357524473099693489668134331674745281535717235257392427302566391 4133003252528032353062743096776231792654135692695969906810496482131665548935603332488960121382005698894406020583821877175245960969563910892001704093580886
0x6c5a69b1b9b340b84c811eeeb2d8bf9da18888a7a09ddced14c54aa57a833bdeb663bf807da974163f35fe379963e98a0bb07df9adfc71504b664ad99b88cd28


#### Klasa użytkowników

In [13]:
#Użytkownik
class User:
    def __init__(self,name,user_id):
        self.name = name
        self.id = user_id #user_id.to_bytes(ID_SIZE//8, byteorder=ENDIAN)
        #self.idn = int.from_bytes(self.id, byteorder=ENDIAN)
        
    def set_private_key(self,key, endian = ENDIAN):
        self.private_key = key
        #self.private_key_n = int.from_bytes(key, endian)
    def generate_ephemeral_exponent(self, ephemeral_exponent_size = EPHEMERAL_EXPONENT_SIZE):
        self.ephemeral_exponent = random.randint(2**(ephemeral_exponent_size//4),2**(ephemeral_exponent_size))
    def calculate_shared_secred(self, msg, id_corr, kgc_public, endian = ENDIAN):
        #print("id_corr", id_corr)
        #hsh = hashH1(corr_idn.to_bytes(kgc_public.key_size//8, endian), kgc_public)
        hsh = hashH1(id_corr, kgc_public)
        rev_hsh = eea(kgc_public.N, hsh)[2]
        
        print("with reversing hsh:", hsh*rev_hsh%kgc_public.N)
        
        
        
        #msgn = int.from_bytes(msg,endian)
        #msgn_e = pow(msgn, kgc_public.e, kgc_public.N)
        msgn_e = pow(msg, kgc_public.e, kgc_public.N)
        
        t = (msgn_e * rev_hsh)%kgc_public.N
        K = pow(t, 2*self.ephemeral_exponent, kgc_public.N)
        
        return K
        
        
    
user_A = User("A",random.randint(1,2**ID_SIZE))
user_B = User("B",random.randint(1,2**ID_SIZE))

#### Generacja kluczy prywatnych uczestników mOT.

In [20]:
# funkcja hashująca do grupy reszt kwadratowych
def generate_user_ltk(kgc, user_id, endian = ENDIAN):
    #context = hashes.Hash(kgc.h1sf, backend=default_backend()) 
    #context.update(material)
    #exponent = int.from_bytes(context.finalize(), byteorder = endian)
    #sk = pow(kgc.QRN_generator, exponent, kgc01.N)
    hsh = hashH1(user_id, kgc.public(), endian)
    print('hsh:\t', hex(hsh))
    return pow(hsh, kgc.d, kgc.N)

user_A.set_private_key(generate_user_ltk(kgc01, user_A.id))
user_B.set_private_key(generate_user_ltk(kgc01, user_B.id))
print((user_A.private_key))
print('{:08x}'.format(user_A.private_key))
    

    

hashH1: number to hash
2959ecb61cfd20e4
hashH1: bytes to hash:
e420fd1cb6ec5929
h_result:	 10d37e6e125ddfcfe79a7393557b95559db6273acc6b3e3b075ad47989cb8182
hsh1:	 11b1e876a2e8c211ff96b999f5a93f61fc889be17c0692edc4cbef1e4c30fb77ccfb2a44ae8c4ebd628ccd286bbbb53beb8b80a448c3404c6732f47c9b04604
hsh:	 0x11b1e876a2e8c211ff96b999f5a93f61fc889be17c0692edc4cbef1e4c30fb77ccfb2a44ae8c4ebd628ccd286bbbb53beb8b80a448c3404c6732f47c9b04604
hashH1: number to hash
be4a593a014e8b73
hashH1: bytes to hash:
738b4e013a594abe
h_result:	 7946680521bbcb61badbc61d78ab523c01803d2f130b946cd35d6b354793978a
hsh1:	 3973a1b1e6f62478871d3f4e06456a6e530b714a5bdbbf99b4036175dd1ae753c51524cec9db2e4b37975ad5eca2f7bb64e8a86173344e65ae114e49ee301664
hsh:	 0x3973a1b1e6f62478871d3f4e06456a6e530b714a5bdbbf99b4036175dd1ae753c51524cec9db2e4b37975ad5eca2f7bb64e8a86173344e65ae114e49ee301664
275293462402902552462520059146324312791083470131727067043397311279449310806698370905309871657359256975222463458816566642023443267722360486344372

##### Weryfikacja klucza 

In [15]:
usk = user_A.private_key
usk_pow_e = pow(usk,kgc01.e,kgc01.N)
#usk_pow_e = usk
hash_context = hashes.Hash(select_SHA_context(H1_EXPONENT_SIZE), backend=default_backend()) 
hash_context.update(user_A.id.to_bytes(ID_SIZE, byteorder = ENDIAN))
exponent = int.from_bytes(hash_context.finalize(), byteorder = ENDIAN)
sk_pre_d = pow(kgc01.QRN_generator, exponent, kgc01.public().N)
print ( sk_pre_d == usk_pow_e)
if(sk_pre_d != usk_pow_e):
    print (usk_pow_e,"\n", sk_pre_d )
    




False
57922494136899980617930693983894451084254714939589074904882957606803938815463230589539576725364335011361448888343046737192049066209068028013626367428100 
 5998558106224773810713346196140367480046953117973350677071102146535280203038132555433957775477682053303435104696817770011164454491679810417275229862601160


## Przebieg sesji

### Generacja wiadomości


In [16]:

def generate_negotiation_message(user, kgc_public, endian = ENDIAN):
    g_p = pow(kgc_public.QRN_generator, user.ephemeral_exponent, kgc_public.N)
    return ((g_p*user.private_key)%kgc_public.N)
    

user_A.generate_ephemeral_exponent()
user_B.generate_ephemeral_exponent()

msg_A = generate_negotiation_message(user_A,kgc01.public())
msg_B = generate_negotiation_message(user_B,kgc01.public())

#print(msg_A)

    

#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="little"))
#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="big"))
print(user_A.id)



#print(idA)
#print(hex(user_A.id))

2979672895396847844


In [17]:
k_A = user_A.calculate_shared_secred(msg_B, user_B.id, kgc01.public())
k_B = user_B.calculate_shared_secred(msg_A, user_A.id, kgc01.public())

k = pow(kgc01.QRN_generator,2*user_A.ephemeral_exponent*user_B.ephemeral_exponent*kgc01.e, kgc01.N)
#print (kgc01.QRN_generator)
print("k=\t", k)
print("k_A=\t", k_A)
print("k_B=\t", k_B)

k1a = pow(msg_B, kgc01.e, kgc01.N)
k2a = eea(kgc01.N,hashH1(user_B.id, kgc01.public()))[2]
k3a = k1a*k2a%kgc01.N
k4a = pow(k3a, 2*user_A.ephemeral_exponent, kgc01.N)
print("k1a =\t", k1a)
print("k2a =\t", k2a)
print("k3a =\t", k3a)
print("k4a =\t", k4a)



hashH1: number to hash
be4a593a014e8b73
hashH1: bytes to hash:
738b4e013a594abe
7946680521bbcb61badbc61d78ab523c01803d2f130b946cd35d6b354793978a
with reversing hsh: 1
hashH1: number to hash
2959ecb61cfd20e4
hashH1: bytes to hash:
e420fd1cb6ec5929
10d37e6e125ddfcfe79a7393557b95559db6273acc6b3e3b075ad47989cb8182
with reversing hsh: 1
k=	 6939412813272280181076292556663959403602458291078502894804199479231806479088627095866996060073362477393921776792203638275016768412941001870943872460665778
k_A=	 6939412813272280181076292556663959403602458291078502894804199479231806479088627095866996060073362477393921776792203638275016768412941001870943872460665778
k_B=	 6939412813272280181076292556663959403602458291078502894804199479231806479088627095866996060073362477393921776792203638275016768412941001870943872460665778
hashH1: number to hash
be4a593a014e8b73
hashH1: bytes to hash:
738b4e013a594abe
7946680521bbcb61badbc61d78ab523c01803d2f130b946cd35d6b354793978a
k1a =	 829808055593091657721809816776964